### Analyse automatique du narratif 
`Shami THIRION SEN`

# Semantique textuelle



- construction de corpus contrastif
  - corpus communiste vs. «non-communiste»

- défi temps - disponibilité des données 
  - si le tp sem 1 visait  le narratifs des temps communiste, (en hind sight) l'obtention du corpus est un défi non négligeable. À l'époque le Bengal est encore loin de la digitalisation des médias.
  

- 

In [1]:
import numpy as np
import json
import glob
from pprint import pprint
from collections import defaultdict
import os, math
from utils import * # import des fonctions depuis le fichier utils

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#visualisation
import pyLDAvis
import pyLDAvis.gensim

# outils TAL en bengali
import BnLemma as lm
from bnlp import BengaliPOS

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
# Bangla POS tagging
bn_pos = BengaliPOS()
from bnlp import BengaliCorpus as corpus
stopwords = corpus.stopwords[:20] + ['রি' + 'টি']

In [3]:
def gen_words(texts):
    single_string = [ ' '.join(line) for line in texts]
    data_words = [gensim.utils.simple_preprocess(text) for text in single_string]
    return data_words


In [4]:
## BIGRAMMES ET TIGRAMMES
def make_bigrams(texts):
    # print(bigram[doc] for doc in texts)
    return ([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return([trigram[bigram[doc]] for doc in texts])


In [5]:
# on choisit le nombre de tokens (pour les 3 corpus)
num_tokens = 10000

#### BanglaGanashakti 
- Lecture du corpus extrait et nettoyage

In [ ]:
banglaGanashakti = "../corpus/txtFiles/banglaGanashakti.txt"
filtered_ganashakti = read_corpus(banglaGanashakti, num_tokens)
filtered_ganashakti = [list for list in filtered_ganashakti if len(list)>0] # old list_of_docs


In [ ]:
# generation   
data_words  = gen_words(filtered_ganashakti)
print(data_words[9][:200])

# copy_this
bigram_phrases = gensim.models.Phrases(filtered_ganashakti, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[filtered_ganashakti], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

data_bigrams = make_bigrams(filtered_ganashakti)
data_bigrams_trigrams = make_trigrams(data_bigrams)

# data_bigrams_trigrams = make_bigrams_trigrams(filtered_ganashakti)

In [ ]:
print(data_bigrams_trigrams[:10])

In [ ]:
### TF-IDF

id2word = Dictionary(data_bigrams_trigrams)
corpus = [id2word.doc2bow(text) for text in filtered_ganashakti]

word = id2word[[9][:1][0]]
print (word)

In [ ]:
corpus=get_corpus(corpus,id2word)
# corpus[:10]
# print(corpus)

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=200,
                                           passes=10,
                                           alpha="auto")

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

In [ ]:
pyLDAvis.save_html(vis, 'ganashaki.html')

#### Calcul spécificité

In [ ]:
max_specificity, min_specificity = get_higest_lowest_specificity(filtered_ganashakti, 10)

In [ ]:
max_specificity, '\n\n', min_specificity

#### Count bigrams

In [ ]:
# count_bigrams(filtered_ganashakti)

In [ ]:
# count_trigrams(filtered_ganashakti)

### Bartamanpatrika

In [ ]:
bartamanpatrika = "../corpus/txtFiles/bartamanpatrika.txt"
filtered_bartamanpatrika = read_corpus(bartamanpatrika, num_tokens)
filtered_bartamanpatrika = [list for list in filtered_bartamanpatrika if len(list)>0]

In [ ]:
# generation   
data_words  = gen_words(filtered_bartamanpatrika)
# print(data_words[9][:200])

# copy_this
bigram_phrases = gensim.models.Phrases(filtered_bartamanpatrika, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[filtered_bartamanpatrika], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

data_bigrams = make_bigrams(filtered_bartamanpatrika)
data_bigrams_trigrams = make_trigrams(data_bigrams)


In [ ]:
print(data_bigrams_trigrams[:10])

In [ ]:
### TF-IDF

id2word = Dictionary(data_bigrams_trigrams)
corpus = [id2word.doc2bow(text) for text in filtered_bartamanpatrika]

word = id2word[[9][:1][0]]
print (word)

In [ ]:
corpus=get_corpus(corpus,id2word)


In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")



In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

In [ ]:
pyLDAvis.save_html(vis, 'bartamanpatrika.html')

#### Calcul de spécificité

In [ ]:
max_specificity, min_specificity = get_higest_lowest_specificity(filtered_bartamanpatrika, 10)
# max_specificity, min_specificity

#### Bigrammes

In [ ]:
# count_bigrams(filtered_bartamanpatrika)

In [ ]:
# count_trigrams(filtered_bartamanpatrika)

### Anandabazar

- corpus qui avec 1000 tokens pointe vers cricket et le sujet de sport 
- Alors qu'avec 10000 nous obtenons plus de généralité

In [ ]:
anandabazar = "../corpus/txtFiles/anandabazar.txt"
filtered_anandabazar = read_corpus(anandabazar, num_tokens)
filtered_anandabazar = [list for list in filtered_anandabazar if len(list)>0] 

In [ ]:
# generation   
data_words  = gen_words(filtered_anandabazar)
# print(data_words[9][:200])

# copy_this
bigram_phrases = gensim.models.Phrases(filtered_anandabazar, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[filtered_anandabazar], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

data_bigrams = make_bigrams(filtered_anandabazar)
data_bigrams_trigrams = make_trigrams(data_bigrams)


In [ ]:
print(data_bigrams_trigrams[:10])

In [ ]:
### TF-IDF

id2word = Dictionary(data_bigrams_trigrams)
corpus = [id2word.doc2bow(text) for text in filtered_anandabazar]

word = id2word[[9][:1][0]]
print (word)

corpus=get_corpus(corpus,id2word)
# corpus[:10]

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=200,
                                           passes=10,
                                           alpha="auto")



In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

In [ ]:
pyLDAvis.save_html(vis, 'anandabazar.html')

#### Calcul de spécificité

In [ ]:
max_specificity, min_specificity = get_higest_lowest_specificity(filtered_anandabazar, 10)

In [ ]:
# max_specificity, min_specificity

#### Bigrammes

In [ ]:
# count_bigrams(filtered_anandabazar)

In [ ]:
# count_trigrams(filtered_anandabazar)